In [11]:
import pandas as pd
import numpy as np

In [12]:
train_set = pd.read_csv("train_ver2.csv")
#test_set = train_set.loc[train_set['fecha_dato']=='2016-04-28', :"segmento"]
test_set = pd.read_csv("test_ver2.csv")

In [13]:
drop_list = ['fecha_alta', 'ult_fec_cli_1t', 'canal_entrada', 'cod_prov', 'indrel_1mes', 'conyuemp']
train_set.drop(drop_list, axis=1, inplace=True)
test_set.drop(drop_list, axis=1, inplace=True)

In [14]:
train_set_505 = train_set.loc[(train_set['fecha_dato']=='2015-05-28')]
train_set_506 = train_set.loc[(train_set['fecha_dato']=='2015-06-28')]
train_set_505.set_index('ncodpers', inplace=True)
train_set_506.set_index('ncodpers', inplace=True)
train_set_june = train_set_505.copy()
train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] = train_set_506.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] - train_set_505.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"]
train_set_june.dropna(inplace=True)
train_set_june['filter'] = train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"].abs().sum(1)
train_set_june = train_set_june[train_set_june['filter']>0]
train_set_june_y = train_set_june.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"].copy().reset_index()
train_set_june_y.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"] = (train_set_june_y.loc[:, "ind_ahor_fin_ult1":"ind_recibo_ult1"]>0).astype(float)
train_set_june = train_set_june.loc[:, :"segmento"]
train_set_june.reset_index(inplace=True)

train_set_june.drop(['fecha_dato'], axis=1, inplace=True)
test_set.drop(['fecha_dato'], axis=1, inplace=True)

In [15]:
product_list = list(train_set.columns)[list(train_set.columns).index("ind_ahor_fin_ult1"):list(train_set.columns).index("ind_recibo_ult1")+1]
def append_last_month(data_set, last_month): 
    return pd.merge(data_set, last_month.loc[:, ['ncodpers']+product_list].drop_duplicates(), how="inner", on='ncodpers')
    
train_set_june = append_last_month(train_set_june, train_set.loc[(train_set['fecha_dato']=='2015-05-28')])
test_set = append_last_month(test_set, train_set.loc[(train_set['fecha_dato']=='2016-05-28')])

In [16]:
province_code = train_set.groupby("nomprov").agg({"renta":np.mean}).sort_values(by='renta', ascending=False).reset_index().reset_index().drop(["renta"], axis=1)
province_code.columns = ['f_nomprov', "nomprov"]

country_code = train_set.groupby('pais_residencia').agg({'renta':np.mean}).sort_values(by='renta', ascending=False).reset_index().reset_index().drop(["renta"], axis=1)
country_code.columns = ['f_pais_residencia', "pais_residencia"]

renta_missing = train_set_june['renta'].mean()

In [17]:
def one_hot_encoding(df, column_name, value_set):
    df1 = df.copy()
    assert len(value_set)>1, "len of value set should >1"
    if len(value_set)==2:
        df1["f_{0}".format(column_name)] = df1[column_name].apply(lambda x: 1 if x==value_set[0] else 0)
    else:
        for v in value_set:
            df1["f_{0}_{1}".format(column_name, v)] = df1[column_name].apply(lambda x: 1 if x==v else 0)
    return df1

In [32]:
import gc

def encode_feature(X):
    X1 = X.copy()
    get_value_set = lambda x: list(train_set[x].drop_duplicates().dropna())
    one_hot_encoding_list = ["ind_empleado", "sexo",  "tiprel_1mes", "indresi", "indext", "indfall", "ind_actividad_cliente", "segmento"]
    for c in one_hot_encoding_list:
        # print c
        X1 = one_hot_encoding(X1, c, get_value_set(c))
        gc.collect()
    copy_feature_list = ['age', 'antiguedad', 'indrel', 'ind_nuevo', 'tipodom']
    for c in copy_feature_list:
        X1['f_'+c] = X1[c]  
    X1 = pd.merge(X1, province_code, how="left", on=['nomprov'])
    X1 = X1.fillna(province_code['f_nomprov'].mean())
    X1 = pd.merge(X1, country_code, how="left", on=['pais_residencia'])
    X1 = X1.fillna(country_code['f_pais_residencia'].mean())
    X1['f_renta'] = (X1['renta'].apply(lambda x: renta_missing if "NA" in str(x) else x)).astype(float)
    X1.drop(one_hot_encoding_list+copy_feature_list+['nomprov', 'pais_residencia', 'renta'], axis=1, inplace=True)
    return X1

train_feature = encode_feature(train_set_june)
test_feature = encode_feature(test_set)

In [33]:
## new
ma = train_feature['f_renta'].max()
mm = train_feature['f_renta'].min()
train_feature['f_renta'] = (train_feature['f_renta'] - mm)/(ma-mm)*2
test_feature['f_renta'] = (test_feature['f_renta'] -mm)/(ma-mm)*2

ma = train_feature['f_nomprov'].max()
mm = train_feature['f_nomprov'].min()
train_feature['f_nomprov'] =(train_feature['f_nomprov']- mm)/(ma-mm)*2
test_feature['f_nomprov'] = (test_feature['f_nomprov']- mm)/(ma-mm)*2

ma = train_feature['f_pais_residencia'].max()
mm = train_feature['f_pais_residencia'].min()
train_feature['f_pais_residencia'] = (train_feature['f_pais_residencia'] - mm)/(ma-mm)*2
test_feature['f_pais_residencia'] = (test_feature['f_pais_residencia'] - mm)/(ma-mm)*2

In [34]:
print len(train_feature)
print len(train_set_june_y)
train_feature = train_feature.sort_values(by='ncodpers')
train_set_june_y = train_set_june_y.sort_values(by='ncodpers')
X_train = np.array(train_feature.iloc[:, 1:].astype(float).values)
y_train = np.array((train_set_june_y.iloc[:, 1:]>0).astype(int).values)
X_test = np.array(test_feature.iloc[:, 1:].astype(float).values)
print len(X_test)

44127
44127
929615


In [35]:
X_train_n = []
y_train_n = []
for i, item in enumerate(X_train):
    for k, y in enumerate(y_train[i]):
        if y>0:
            X_train_n.append(item)
            y_train_n.append(k)
X_train_n = np.array(X_train_n)
y_train_n = np.array(y_train_n)

In [36]:
'''
mu = X_train_n.mean(0)
std = X_train_n.std(0)
idx = np.argwhere(std==0)
X_train_n = np.delete(X_train_n, idx, axis=1)
X_test = np.delete(X_test, idx, axis=1)
mu = X_train_n.mean(0)
std = X_train_n.std(0)

X_train_n = (X_train_n-mu)/std
X_test = (X_test-mu)/std
'''
std = X_train_n.std(0)
idx = np.argwhere(std==0)
X_train_n = np.delete(X_train_n, idx, axis=1)
X_test = np.delete(X_test, idx, axis=1)

array([[ -7.43304413e-03,  -1.28751215e-02,   1.18011168e+00, ...,
         -7.01053318e-01,   5.25588331e-03,   7.15122699e-01],
       [ -7.43304413e-03,  -1.28751215e-02,   1.18011168e+00, ...,
          1.62057375e+00,   5.25588331e-03,   1.06238049e-03],
       [ -7.43304413e-03,  -1.28751215e-02,   1.18011168e+00, ...,
          2.47003725e-03,   5.25588331e-03,   1.06238049e-03],
       ..., 
       [ -7.43304413e-03,  -1.28751215e-02,   1.18011168e+00, ...,
          6.35641057e-01,   5.25588331e-03,  -2.75617089e-01],
       [ -7.43304413e-03,  -1.28751215e-02,   1.18011168e+00, ...,
          6.35641057e-01,   5.25588331e-03,   1.65827910e-02],
       [ -7.43304413e-03,  -1.28751215e-02,  -8.47377426e-01, ...,
          2.11304010e+00,   5.25588331e-03,   1.06238049e-03]])

In [ ]:
train_set = pd.read_csv("../data/train_ver2.csv")

In [37]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight=None, multi_class='multinomial', solver="lbfgs", penalty='l2')
model = model.fit(X_train_n, y_train_n)

In [38]:
predict = model.predict_proba(X_test)

In [39]:
predict = np.hstack([np.zeros((predict.shape[0],2)), predict])

In [28]:
model.classes_

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23])

In [40]:
filter1 = np.array((test_feature.loc[:, product_list]==0).values)
predict = predict*filter1
user_id = list(test_set["ncodpers"])

f = open("softmax_sub.csv", 'w')
f.write("ncodpers,added_products\n")

print len(predict)

for idx,r in enumerate(predict):
    order = [i[0] for i in sorted(enumerate(r), key=lambda x:x[1], reverse=True)]
    select = order[:min(7,len(order))]
    f.write(str(user_id[idx])+","+(" ".join([product_list[s] for s in select]))+'\n')
f.close()

929615


In [8]:
train_set_june.sum()

ncodpers                                                       30991704659
ind_empleado             NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
pais_residencia          ESESESESESESESESESESESESESESESESESESESESESESES...
sexo                     VHVVVVHHVHHVVHVHVHHVVHHVHVVVVVHHHVHVVHHVVHHVVV...
age                       24 27 23 40 24 23 24 31 23 23 31 25 35 24 26 ...
ind_nuevo                                                             2356
antiguedad                    34     34     34     34     34     34    ...
indrel                                                               44127
tiprel_1mes              AAIAAAAAAIAAAAAAIAAAAAAAAAAAAAAAAAAAAIAAAAAAAA...
indresi                  SSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS...
indext                   NNNNNNNNNSNNNNNNNSNNNNNNNNNNNNNNNNNNNNNNSNNSNN...
indfall                  NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN...
tipodom                                                              44127
nomprov                  

In [10]:
f = open("simple_sub.csv", 'w')
f.write("ncodpers,added_products\n")
for code in list(test_set['ncodpers']):
    f.write(str(code)+",ind_recibo_ult1 ind_cco_fin_ult1 ind_cno_fin_ult1 ind_ecue_fin_ult1 ind_nom_pens_ult1 ind_tjcr_fin_ult1 ind_nomina_ult1\n")
f.close()